<a href="https://colab.research.google.com/github/UNT-CSCE-Project/sentiment-classification-of-amazon-reviews/blob/dev_avijeet/Copy_of_sentiment_classification_amazon_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install transformers -q

In [5]:

### Connect with google drive

### import csv from google dir
def connect_and_fetch_file():
  from google.colab import drive
  drive.mount('/content/drive')

  file_id = "1ZBezHHtxu8L6pZVeOtaC0-_n3MjOYnGR"

  file_dir = f'https://drive.google.com/uc?id={file_id}'
  return file_dir
file_dir = connect_and_fetch_file()



Mounted at /content/drive


In [6]:
import pandas as pd

df = pd.read_csv(file_dir)


<ipython-input-6-d4830fe90155>:3: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_dir)


In [7]:
df_updated = df.copy()  # Create a copy to keep the original DataFrame
df_updated.columns = [col[len('reviews.'):] if col.startswith('reviews.') else col for col in df.columns]


In [8]:
df_updated=df_updated[[ 'doRecommend',  'numHelpful', 'rating',  'text', 'title']]

In [9]:
from textblob import TextBlob

def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

def sentiment_scores(text):
    sentiment = analyze_sentiment(text)
    if sentiment > 0.5:
        return 5
    elif 0.2 < sentiment <= 0.5:
        return 4
    elif -0.2 <= sentiment <= 0.2:
        return 3
    elif -0.5 <= sentiment < -0.2:
        return 2
    else:
        return 1



In [10]:
sentiments = df_updated[df_updated['doRecommend'].isna() & df_updated['rating'].isna()]['text'].apply(sentiment_scores)

In [11]:
df_updated['rating'].fillna(df_updated[df_updated['doRecommend'].isna() & df_updated['rating'].isna()]['text'].apply(sentiment_scores), inplace=True)

In [12]:
df_updated['text' ].fillna('Five Stars', inplace=True)

In [13]:
df_updated['doRecommend'].fillna(False, inplace=True)
df_updated['numHelpful'].fillna(0, inplace=True)

In [14]:
df_updated[df_updated['title'].isna()].iloc[:]

,doRecommend,numHelpful,rating,text,title
3083,False,0.0,5.0,A must have item.,NaN
21953,True,1.0,5.0,I like it. I use mostly for music.,NaN
25515,True,0.0,5.0,I was totally stoked about this product becaus...,NaN
28578,True,0.0,5.0,I'm in LOVE with ALEXA!!,NaN
28680,True,1.0,5.0,"Love it ! I started with a kindle,then a fire ...",NaN


In [15]:
df_updated['numHelpful'] = df_updated['numHelpful'].astype('int')
df_updated['rating'] = df_updated['rating'].astype('int')

In [16]:
df_updated = df_updated.drop('title', axis=1)

In [21]:

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [26]:
pip install transformers torch


## Contradictions

In [18]:
contractions_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot",
                   "can't've": "cannot have", "'cause": "because", "could've": "could have",
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not",
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will",
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did",
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                   "I'll've": "I will have","I'm": "I am", "I've": "I have",
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                   "i'll've": "i will have","i'm": "i am", "i've": "i have",
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                   "it'll": "it will", "it'll've": "it will have","it's": "it is",
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                   "she's": "she is", "should've": "should have", "shouldn't": "should not",
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is",
                   "there'd": "there would", "there'd've": "there would have","there's": "there is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                   "they'll've": "they will have", "they're": "they are", "they've": "they have",
                   "to've": "to have", "wasn't": "was not", "we'd": "we would",
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                   "we're": "we are", "we've": "we have", "weren't": "were not",
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                   "what's": "what is", "what've": "what have", "when's": "when is",
                   "when've": "when have", "where'd": "where did", "where's": "where is",
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have",
                   "who's": "who is", "who've": "who have", "why's": "why is",
                   "why've": "why have", "will've": "will have", "won't": "will not",
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [19]:
import re
contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
    if isinstance(s, str):
        def replace(match):
            return contractions_dict[match.group(0)]

        return contractions_re.sub(replace, s)
    else:
        return s

In [20]:
df_updated['expanded_text']  = df_updated['text'].apply(expand_contractions)

In [42]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", tokenizer="distilbert-base-cased")

In [ ]:

threshold = .3
questions = [
    "Do you love it?",
    "How good is it?",
    "How expensive is it?",
    "Do you recommend it?"
]

def match_questions_to_reviews(review, questions):
    matches = []
    for question in questions:
        result = qa_pipeline(question=question, context=review)
        matches.append((question, result["answer"], result["score"]))
    return matches

def check_reviews(review):
    question_matches = match_questions_to_reviews(review, questions)

    for question, answer, score in question_matches:

        if score >=threshold:
          return True

    return False


df_updated['Taken'] = df_updated['expanded_text'].apply(check_reviews)


In [46]:
df_updated.to_csv('/content/drive/MyDrive/PhD/NLP/final/qa_reviews.csv')

0      True
1      True
2      True
3      True
4      True
5     False
6     False
7     False
8      True
9      True
10     True
11     True
12     True
13    False
14     True
15     True
16     True
17     True
18     True
19     True
Name: Taken, dtype: object

## After Question Answer


In [22]:
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def process_text(text):
    if isinstance(text, str):
        sentences = sent_tokenize(text)

        stop_words = set(stopwords.words('english'))
        result_text = [
            ' '.join(
                WordNetLemmatizer().lemmatize(word.lower())
                for word in word_tokenize(sentence) if word.lower() not in stop_words and
            )
            for sentence in sentences
        ]


        result_text = ' '.join(result_text)

        return result_text
    else:
        return ''


In [25]:
df_updated['process_text'] = df_updated['expanded_text'].apply(process_text)

# references


* https://towardsdatascience.com/text-normalization-for-natural-language-processing-nlp-70a314bfa646


